In [ ]:
%autosave 60

# The Davis-Putnam Algorithm

This notebook implements the algorithm of Davis and Putnam.  Further details are given in the lecture notes.

The function $\texttt{negateLiteral}(L)$ computes the complement of a literal $L$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{negateLiteral}(p) = \neg p$,
    </li>
    <li>$\texttt{negateLiteral}(\neg p) = p$.
    </li>
</ol>

In [ ]:
def negateLiteral(L):
    "Compute the complement of the literal L."
    if isinstance(L, str):
        return ('¬', L)
    else:
        return L[1]

In [ ]:
negateLiteral('p')

In [ ]:
negateLiteral(('¬', 'p'))

The function $\texttt{extractVariable}(L)$ extracts the variable from the literal $L$.
If $p$ is a propositional variable, we have the following: 
<ol>
    <li>$\texttt{extractVariable}(p) = p$,
    </li>
    <li>$\texttt{extractVariable}(\neg p) = p$.
    </li>
</ol>

In [ ]:
def extractVariable(L):
    "Extract the propositional variable from the literal L."
    if isinstance(L, str):
        return L
    else:
        return L[1]

The function $\texttt{arb}(S)$ returns an arbitrary element from the set $S$.

In [ ]:
def arb(S):
    "Return some member from the set S."
    for x in S:
        return x

We select an arbitrary literal from an arbitrary clause from the set <tt>Clauses</tt> that is not a member of the set of variables in the set <tt>Forbidden</tt>.

In [ ]:
def selectLiteral(Clauses, Forbidden):
    Variables = { extractVariable(L) for C in Clauses for L in C } - Forbidden
    return arb(Variables)

Given a set of clauses $\textttt{C}$ and a literal $L$, the procedure $\texttt{reduce}(C, L)$ performs all unit cuts and all unit subsumptions on clauses of $C$ that are possible using the literal $L$.  The resulting set of clauses is returned.

In [ ]:
def reduce(Clauses, L):
    LBar = negateLiteral(L)
    return   { C - { LBar } for C in Clauses if LBar in C }          \
           | { C for C in Clauses if LBar not in C and L not in C }  \
           | { frozenset({L}) }

In [ ]:
L = ('-','p')
{ frozenset({L}) }

$\texttt{Clauses}$ is a set of clauses.  The call $\texttt{saturate}(\texttt{Clauses})$ computes the set of those clauses that can be derived from $S$ via unit cuts.  Furthermore, all clauses in S that are subsumed by unit clauses are removed from $S$.

In [ ]:
def saturate(Clauses):
    S     = Clauses.copy()
    Units = { C for C in S if len(C) == 1 }
    Used  = set()
    while len(Units) > 0:
        unit  = Units.pop()
        Used |= { unit }
        L     = arb(unit)
        S     = reduce(S, L)
        Units = { C for C in S if len(C) == 1 } - Used        
    return S

The function $\texttt{solve}(\texttt{Clauses}, \texttt{Literals})$ takes a set of clauses and a set of Literals as input.  The function tries to compute a variable assignment that makes all of the clauses true.  If this is successful, a set of unit clauses is returned.  This set of unit clauses does not contain  any complementary literals and therefore corresponds to a variable assignment satisfying all clauses.  If clauses is unsatisfiable, the set containing the  empty clause is returned.

The argument literals contains all those literals that have already been used  to reduce the clauses.  Initially, this set is empty.

In [ ]:
def solve(Clauses, Literals):

    S      = saturate(Clauses);
    empty  = frozenset()
    falsum = {empty}
    if empty in S:
        return falsum
    if all(len(C) == 1 for C in S):
        return S
    L      = selectLiteral(S, Literals)
    negL   = negateLiteral(L)
    Result = solve(S | { frozenset({L}) }, Literals | { L })
    if Result != falsum:
        return Result
    return solve(S | { frozenset({negL}) }, Literals | { L })

The function $\texttt{toString}(S)$ takes a set $S$ as input.  The set $S$ is a set of frozensets and the function converts $S$ into a string that looks like a set of sets. 

In [ ]:
def toString(S):
    "Convert the set S of frozen sets to a string."
    if S == set():
        return '{}'
    result = '{ '
    for f in S:
        result += str(set(f)) + ', '
    result = result[:-2]
    result += ' }'
    return result
    

In [ ]:
c1 = frozenset({ 'r', 'p', 's' })
c2 = frozenset({ 'r', 's' })
c3 = frozenset({ 'p', 'q', 's' })
c4 = frozenset({ ('¬', 'p'), ('¬', 'q') })
c5 = frozenset({ ('¬', 'p'), 's', ('¬', 'r') })
c6 = frozenset({ 'p', ('¬', 'q'), 'r'})
c7 = frozenset({ ('¬', 'r'), ('¬', 's'), 'q' })
c8 = frozenset({ ('¬', 'p'), ('¬', 's')})
c9 = frozenset({ 'p', ('¬', 'r'), ('¬', 'q') })
c0 = frozenset({ ('¬', 'p'), 'r', 'q', ('¬', 's') })
S  = { c0, c1, c2, c3, c4, c5, c6, c7, c8, c9 }
toString(solve(S, set()))